<a href="https://colab.research.google.com/github/nmanhthi7111994/AI_machine_learning/blob/main/player_detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Attemp to validate the training test

In [1]:
%cd /content
!pip install -q roboflow
!pip install -q unzip
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Cloning into 'ultralytics'...
remote: Enumerating objects: 41838, done.
remote: Counting objects: 100% (576/576), done.
remote: Compressing objects: 100% (369/369), done.
remote: Total 41838 (delta 354), reused 373 (delta 202), pack-reused 41262 (from 1)
Receiving objects: 100% (41838/41838), 31.43 MiB | 39.49 MiB/s, done.
Resolving deltas: 100% (30989/30989), done.
/content/ultralytics
Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
 

In [ ]:
%cd /content
!curl -L "https://universe.roboflow.com/ds/I0Xj0fADqQ?key=IEi1HcEtIZ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cd /content
!wget http://14.225.204.171:9998/training_3_70_epoch_20.pt

In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from contextlib import redirect_stdout
import logging
from google.colab.patches import cv2_imshow

# Define paths
model_path = '/content/training_3_70_epoch_20.pt'  # Path to your trained model
input_folder = '/content/test/images'           # Path to the folder containing images
label_folder = '/content/test/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/result/1st_model_evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/result/1st_model_final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []
image_with_detections_count = 0  # Counter for images with detections

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '0': 'ball',
                '1': 'goalkeeper',
                '2': 'football-players',
                '3': 'referee',
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Suppress detailed output from YOLO model predictions
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):  # Suppress unwanted output
        for image_name in os.listdir(input_folder):
            # Full path to the image
            image_path = os.path.join(input_folder, image_name)

            # Check if the file is an image
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                continue

            # Perform detection
            results = model.predict(source=image_path, verbose=False)

            # Extract detected class names
            detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

            # If detections exist, increment the counter
            if detected_classes:
                image_with_detections_count += 1

            # Corresponding label file
            label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
            label_path = os.path.join(label_folder, label_file)

            # Parse ground truth classes from label file
            if os.path.exists(label_path):
                gt_classes = parse_label_file(label_path)
            else:
                gt_classes = []

            # Compare detections with ground truth classes
            tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
            fp = len(detected_classes) - tp  # False Positives: incorrect class detections
            fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

            # Save results
            results_data.append({
                'Image': image_name,
                'True Positives': tp,
                'False Positives': fp,
                'False Negatives': fn,
                'Detected Classes': detected_classes,
                'Ground Truth Classes': gt_classes
            })

            # Save the output image with detections
            detected_img = results[0].plot()
            output_path = os.path.join(output_folder, f"detected_{image_name}")
            cv2.imwrite(output_path, detected_img)
            # Uncomment the line below if you want to display the image in Colab
            cv2_imshow(detected_img)

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)

# Print the number of images with detections
print(f"Total number of images with detections: {image_with_detections_count}")

In [4]:
!yolo task=detect mode=train model=yolov8n.pt data=/content/data.yaml epochs=100 imgsz=640 batch=32

import os
import glob
from google.colab import files

# Define the base path where YOLO saves the training results
base_path = "/content/ultralytics/runs/detect/"

# Get a list of all training folders
train_folders = glob.glob(os.path.join(base_path, 'train*'))

# Find the most recently modified training folder
latest_train_folder = max(train_folders, key=os.path.getmtime)

# Construct the path to the best model
model_path = os.path.join(latest_train_folder, "weights", "best.pt")

# Check if the model file exists and download it
if os.path.exists(model_path):
    print(f"Training completed. Model found in {latest_train_folder}. Preparing to download...")
    files.download(model_path)
else:
    print("Model not found. Please check the training process and paths.")

100% 6.25M/6.25M [00:00<00:00, 74.9MB/s]
Ultralytics YOLOv8.2.98 🚀 Python-3.10.12 torch-2.4.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>